In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [73]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input 1x28x28 OUtput32x28x28 RF 3x3
        self.bn0=nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)#input 32x28x28 OUtput 64x28x28 RF 5x5
        self.bn1=nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2)#input 64x28x28 OUtput 64x14x14 RF 10x10
        self.drop=nn.Dropout(0.25)
        self.conv3_1 = nn.Conv2d(32, 16, 1, padding=1)
        self.bn2=nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(16, 16, 3, padding=1)#input 64x14x14 OUtput 128x14x14 RF 12x12
        self.bn3=nn.BatchNorm2d(16)
        self.conv4 = nn.Conv2d(16, 16, 3, padding=1)#input 128x14x14 OUtput 256x14x14 RF 14x14
        self.bn4=nn.BatchNorm2d(16)
        self.pool2 = nn.MaxPool2d(2, 2)#input -256x14x14 OUtput256x7x7 RF 28x28
        self.conv5 = nn.Conv2d(16, 16, 3, padding=1)
        self.bn5=nn.BatchNorm2d(16)
        self.conv6 = nn.Conv2d(16, 10, 3, padding=1)
        #self.conv5 = nn.Conv2d(256, 512, 3)#input -256x7x7 OUtput 512x5x5 RF 30x30
        #self.conv6 = nn.Conv2d(512, 1024, 3)#input 512x5x5 OUtput 1024x3x3  RF 32x32
        #self.conv7 = nn.Conv2d(1024, 10, 3)#input 1024x3x3 OUtput 10x1x1 RF 34x34
        #self.fc1 = nn.Linear(64,128)
        #self.fc2 = nn.Linear(128,10)
    def forward(self, x):
        x = self.drop(self.pool1(self.bn1(F.relu(self.conv2(self.bn0(F.relu(self.conv1(x))))))))
        #print(x.shape)
        x = self.drop(self.pool2(self.bn4(F.relu(self.conv4(self.bn3(F.relu(self.conv3(self.bn2(F.relu(self.conv3_1(x)))))))))))
        #print(x.shape)
        x = F.adaptive_avg_pool2d(self.conv6(self.bn5(F.relu(self.conv5(x)))), output_size=1)
        #print(x.shape)
        #x=x.view(-1, 64)
        #x = self.fc2(F.relu(self.fc1(x)))
        '''
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = self.conv7(x)
        '''
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [74]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
       BatchNorm2d-2           [-1, 32, 28, 28]              64
            Conv2d-3           [-1, 32, 28, 28]           9,248
       BatchNorm2d-4           [-1, 32, 28, 28]              64
         MaxPool2d-5           [-1, 32, 14, 14]               0
           Dropout-6           [-1, 32, 14, 14]               0
            Conv2d-7           [-1, 16, 16, 16]             528
       BatchNorm2d-8           [-1, 16, 16, 16]              32
            Conv2d-9           [-1, 16, 16, 16]           2,320
      BatchNorm2d-10           [-1, 16, 16, 16]              32
           Conv2d-11           [-1, 16, 16, 16]           2,320
      BatchNorm2d-12           [-1, 16, 16, 16]              32
        MaxPool2d-13             [-1, 16, 8, 8]               0
          Dropout-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [75]:


torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [76]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [77]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0789, Accuracy: 9759/10000 (97.59%)



  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0469, Accuracy: 9863/10000 (98.63%)



  0%|          | 0/938 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0212, Accuracy: 9932/10000 (99.32%)



  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0190, Accuracy: 9939/10000 (99.39%)



loss=0.026329223066568375 batch_id=937: 100%|██████████| 938/938 [00:17<00:00, 52.36it/s]



Test set: Average loss: 0.0226, Accuracy: 9928/10000 (99.28%)

